# Train LLMs for rap generation

Check runtime is set to GPU. Hopefully Tesla T4 shows up below.

In [1]:
!nvidia-smi

Fri Mar 31 09:44:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers datasets
from google.colab import drive
drive.mount('./mydata')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.3 MB/s eta 0:00:00
Mounted at ./myd

## Get train and test sets

In [3]:
import pandas as pd

In [4]:
from datasets import Dataset
train_df = pd.read_csv('mydata/MyDrive/train.csv')
test_df = pd.read_csv('mydata/MyDrive/test.csv')
train_lyrics = [lyric for lyric in list(train_df['lyric']) if isinstance(lyric, str)]
test_lyrics = [lyric for lyric in list(test_df['lyric']) if isinstance(lyric, str)]
train = Dataset.from_dict({'text': train_lyrics})
test = Dataset.from_dict({'text': test_lyrics})
print(train, test)

Dataset({
    features: ['text'],
    num_rows: 16824
}) Dataset({
    features: ['text'],
    num_rows: 4208
})


## Baseline

### Strong baseline: GPT2-large

In [15]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

device = "cuda"
model_id = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [16]:
from datasets import load_dataset
datasets = load_dataset("csv", data_files={
    "train": 'mydata/MyDrive/train.csv', "validation": 'mydata/MyDrive/test.csv'
    })
datasets = datasets.filter(lambda example: isinstance(example["lyric"], str))
datasets

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 4208
    })
})

In [17]:
def tokenize_function(examples):
    return tokenizer(examples['lyric'], truncation=True, max_length=1024)

In [18]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["artist", "song", "lyric"])
tokenized_datasets

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4208
    })
})

In [19]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [20]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

In [21]:
from transformers import Trainer, TrainingArguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-not_trained",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    num_train_epochs=2, 
)
# Don't train, just define to get perplexity

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [23]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 72.25


### Weak baseline: Distilled GPT2

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

In [6]:
from datasets import load_dataset
datasets = load_dataset("csv", data_files={
    "train": 'mydata/MyDrive/train.csv', "validation": 'mydata/MyDrive/test.csv'
    })
datasets = datasets.filter(lambda example: isinstance(example["lyric"], str))
datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7957b17e5f9de427/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Filter:   0%|          | 0/16877 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4220 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 4208
    })
})

In [7]:
def tokenize_function(examples):
    return tokenizer(examples['lyric'], truncation=True, max_length=1024)

In [8]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["artist", "song", "lyric"])
tokenized_datasets

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4208
    })
})

In [9]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

In [12]:
from transformers import Trainer, TrainingArguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-not_trained",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    num_train_epochs=2, 
)
# Don't train, just define to get perplexity

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [14]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 132.44


## Fine-tuning

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained('mydata/MyDrive/model').to(device)

In [ ]:
from datasets import load_dataset
datasets = load_dataset("csv", data_files={
    "train": 'mydata/MyDrive/train.csv', "validation": 'mydata/MyDrive/test.csv'
    })
datasets = datasets.filter(lambda example: isinstance(example["lyric"], str))
datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7957b17e5f9de427/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Filter:   0%|          | 0/16877 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4220 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['artist', 'song', 'lyric'],
        num_rows: 4208
    })
})

In [ ]:
datasets['train']['lyric'][0]

'knockin doors down showin parts around\nima come through and show my wood pine\nknockin knockin doors down showin parts around\nima ima come through and show my wood pine ima ima come\nknockin doors knockin knockin doors down down\nknockin doors knock knock knock knock knockin doors down\nimaima come through ima ima come down\nima come through ima ima come down \npaper chasin on occasion\nwest deer park thats my location\nlife and death what we was facin\nboy gotta get that motivation\nback in the basement\nlook at my idols im feelin adjacent\ndoin the shit that i wish i was doing im feelin impatient\nthrowin parties and catching cases\nthats the life but that shit basic\nive cooked crack i cant erase it\nif i could would not replace it\ngrew up all alone\nhad to teach myself to tie my laces\nwhats the deal\nima play the field\nrattpack we keep it real\nnothin but love for the people that paved the way\nso a brother like me he can get it today\nantoine erykah and 3k\ncant forget those

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['lyric'], truncation=True, max_length=1024)

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["artist", "song", "lyric"])
tokenized_datasets

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 16824
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4208
    })
})

In [ ]:
tokenizer.decode(tokenized_datasets['train']['input_ids'][0])

'knockin doors down showin parts around\nima come through and show my wood pine\nknockin knockin doors down showin parts around\nima ima come through and show my wood pine ima ima come\nknockin doors knockin knockin doors down down\nknockin doors knock knock knock knock knockin doors down\nimaima come through ima ima come down\nima come through ima ima come down \npaper chasin on occasion\nwest deer park thats my location\nlife and death what we was facin\nboy gotta get that motivation\nback in the basement\nlook at my idols im feelin adjacent\ndoin the shit that i wish i was doing im feelin impatient\nthrowin parties and catching cases\nthats the life but that shit basic\nive cooked crack i cant erase it\nif i could would not replace it\ngrew up all alone\nhad to teach myself to tie my laces\nwhats the deal\nima play the field\nrattpack we keep it real\nnothin but love for the people that paved the way\nso a brother like me he can get it today\nantoine erykah and 3k\ncant forget those

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/16824 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4208 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-rap1",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    num_train_epochs=2, 
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.637900,3.748195
2,3.606500,3.712628


TrainOutput(global_step=22560, training_loss=3.582404184003248, metrics={'train_runtime': 5138.2216, 'train_samples_per_second': 35.122, 'train_steps_per_second': 4.391, 'total_flos': 5894397424041984.0, 'train_loss': 3.582404184003248, 'epoch': 2.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 40.96


In [ ]:
trainer.save_model('mydata/MyDrive/model')

# Evaluation

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install eng_to_ipa syllables

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 KB 66.2 MB/s eta 0:00:00
  Created wheel for eng_to_ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822631 sha256=3daa7bcfdf9f4eac0f64cc2262f5ec365dd0c26e505535debc72594a5a8be4f2
  Stored in directory: /root/.cache/pip/wheels/a6/45/86/8db1ee34203342879ae7096b519d62e9fddeade95e16d60362
Successfully built eng_to_ipa
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.1.0
    Uninstalling importlib-metadata-6.1.0:
      Successfully uninstalled importlib-metadata-6.1.0


In [ ]:
import lyrics

In [ ]:
import importlib
importlib.reload(lyrics)

<module 'lyrics' from '/content/lyrics.py'>

In [ ]:
first_lines = [song[:song.index('\n')+1] for song in test_df['lyric'] if isinstance(song, str) and '\n' in song]

In [ ]:
first_lines[0]

'godfathers in the house\n'

## Our fine-tuned model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", use_fast=True)
model = AutoModelForCausalLM.from_pretrained('mydata/MyDrive/mlp/model').to(device)

In [ ]:
import torch
torch.manual_seed(0)

In [ ]:
rd = []
scd = []
lr = []
uw = []
i = 1
for first_line in first_lines:
  if i % 20 == 1:
    print(f"Processing {i}/{len(first_lines)}")
  i += 1

  input_ids = tokenizer([first_line], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  rd.append(lyrics.get_rhyme_density(o))
  scd.append(lyrics.get_syllable_count_difference(o))
  lr.append(lyrics.get_longest_rhyme(o))
  uw.append(lyrics.get_unique_words(o))

Processing 1/4200
Processing 21/4200
Processing 41/4200
Processing 61/4200
Processing 81/4200
Processing 101/4200
Processing 121/4200
Processing 141/4200
Processing 161/4200
Processing 181/4200
Processing 201/4200


Input length of input_ids is 105, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 221/4200
Processing 241/4200


Input length of input_ids is 114, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 261/4200
Processing 281/4200
Processing 301/4200
Processing 321/4200


...

Processing 3001/4200
Processing 3021/4200
Processing 3041/4200
Processing 3061/4200
Processing 3081/4200
Processing 3101/4200
Processing 3121/4200
Processing 3141/4200
Processing 3161/4200
Processing 3181/4200
Processing 3201/4200
Processing 3221/4200
Processing 3241/4200
Processing 3261/4200
Processing 3281/4200
Processing 3301/4200
Processing 3321/4200
Processing 3341/4200
Processing 3361/4200
Processing 3381/4200
Processing 3401/4200
Processing 3421/4200
Processing 3441/4200


Input length of input_ids is 216, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 3461/4200
Processing 3481/4200
Processing 3501/4200
Processing 3521/4200
Processing 3541/4200
Processing 3561/4200
Processing 3581/4200
Processing 3601/4200
Processing 3621/4200
Processing 3641/4200
Processing 3661/4200
Processing 3681/4200


Input length of input_ids is 143, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 3701/4200
Processing 3721/4200
Processing 3741/4200
Processing 3761/4200
Processing 3781/4200
Processing 3801/4200
Processing 3821/4200
Processing 3841/4200
Processing 3861/4200
Processing 3881/4200
Processing 3901/4200
Processing 3921/4200
Processing 3941/4200
Processing 3961/4200
Processing 3981/4200
Processing 4001/4200
Processing 4021/4200
Processing 4041/4200
Processing 4061/4200
Processing 4081/4200
Processing 4101/4200
Processing 4121/4200
Processing 4141/4200
Processing 4161/4200
Processing 4181/4200


In [ ]:
import numpy as np
rd = np.array(rd)
scd = np.array(scd)
lr = np.array(lr)
uw = np.array(uw)

In [ ]:
print(round(np.mean(rd),2), round(np.std(rd),2))

0.54 0.24


In [ ]:
print(round(np.mean(scd),2), round(np.std(scd),2))

2.2 4.57


In [ ]:
print(round(np.mean(lr),2), round(np.std(lr),2))

3.3 1.13


In [ ]:
print(round(np.mean(uw),2), round(np.std(uw),2))

0.51 0.15


## Baseline DistilGPT2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

In [ ]:
import torch
torch.manual_seed(0)

In [ ]:
prompt = '(Below are ONLY lyrics from a rap song)\n\n'

In [ ]:
# rd = []
# scd = []
# lr = []
uw = []
i = 1
for first_line in first_lines:
  if i % 20 == 1:
    print(f"Processing {i}/{len(first_lines)}")
  i += 1

  input_ids = tokenizer([prompt + first_line], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  o = o.replace(prompt, '')
  # rd.append(lyrics.get_rhyme_density(o))
  # scd.append(lyrics.get_syllable_count_difference(o))
  # lr.append(lyrics.get_longest_rhyme(o))
  try:
    uw.append(lyrics.get_unique_words(o))
  except ZeroDivisionError:
    continue

Processing 1/4200
Processing 21/4200
Processing 41/4200
Processing 61/4200
Processing 81/4200
Processing 101/4200
Processing 121/4200
Processing 141/4200
Processing 161/4200
Processing 181/4200
Processing 201/4200


Input length of input_ids is 117, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 221/4200
Processing 241/4200


Input length of input_ids is 126, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 261/4200
Processing 281/4200
Processing 301/4200


...

Input length of input_ids is 132, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 3001/4200
Processing 3021/4200
Processing 3041/4200
Processing 3061/4200
Processing 3081/4200
Processing 3101/4200
Processing 3121/4200
Processing 3141/4200
Processing 3161/4200
Processing 3181/4200
Processing 3201/4200
Processing 3221/4200
Processing 3241/4200
Processing 3261/4200
Processing 3281/4200
Processing 3301/4200
Processing 3321/4200
Processing 3341/4200
Processing 3361/4200
Processing 3381/4200
Processing 3401/4200
Processing 3421/4200
Processing 3441/4200


Input length of input_ids is 228, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 3461/4200
Processing 3481/4200
Processing 3501/4200
Processing 3521/4200
Processing 3541/4200
Processing 3561/4200
Processing 3581/4200
Processing 3601/4200
Processing 3621/4200
Processing 3641/4200


ZeroDivisionError: ignored

In [ ]:
import numpy as np
rd = np.array(rd)
scd = np.array(scd)
lr = np.array(lr)
uw = np.array(uw)

In [ ]:
print(round(np.mean(rd),2), round(np.std(rd),2))

0.55 0.37


In [ ]:
print(round(np.mean(scd),2), round(np.std(scd),2))

5.26 8.77


In [ ]:
print(round(np.mean(lr),2), round(np.std(lr),2))

3.01 1.44


In [ ]:
print(round(np.mean(uw),2), round(np.std(uw),2))

0.59 0.22


## Baseline GPT2-large

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

device = "cuda"
model_id = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [ ]:
import torch
torch.manual_seed(0)

In [ ]:
prompt = '(Below are ONLY lyrics from a rap song)\n'

In [ ]:
rd = []
scd = []
lr = []
uw = []
i = 1
for first_line in first_lines[:1000]:
  if i % 20 == 1:
    print(f"Processing {i}/{len(first_lines)}")
  i += 1

  input_ids = tokenizer([prompt + first_line], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  o = o.replace(prompt, '')
  rd.append(lyrics.get_rhyme_density(o))
  scd.append(lyrics.get_syllable_count_difference(o))
  lr.append(lyrics.get_longest_rhyme(o))
  uw.append(lyrics.get_unique_words(o))

Processing 1/4200
Processing 21/4200
Processing 41/4200
Processing 61/4200
Processing 81/4200
Processing 101/4200
Processing 121/4200
Processing 141/4200
Processing 161/4200
Processing 181/4200
Processing 201/4200


Input length of input_ids is 116, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 221/4200
Processing 241/4200


Input length of input_ids is 125, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 261/4200
Processing 281/4200
Processing 301/4200


...

Processing 341/4200
Processing 361/4200
Processing 381/4200
Processing 401/4200
Processing 421/4200
Processing 441/4200
Processing 461/4200


Input length of input_ids is 120, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 481/4200
Processing 501/4200
Processing 521/4200
Processing 541/4200
Processing 561/4200
Processing 581/4200
Processing 601/4200
Processing 621/4200
Processing 641/4200
Processing 661/4200
Processing 681/4200
Processing 701/4200
Processing 721/4200
Processing 741/4200
Processing 761/4200
Processing 781/4200
Processing 801/4200
Processing 821/4200
Processing 841/4200
Processing 861/4200
Processing 881/4200
Processing 901/4200


Input length of input_ids is 131, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Processing 921/4200
Processing 941/4200
Processing 961/4200
Processing 981/4200


In [ ]:
import numpy as np
print(round(np.mean(rd),2), round(np.std(rd),2))

0.56 0.32


In [ ]:
print(round(np.mean(scd),2), round(np.std(scd),2))

3.19 7.19


In [ ]:
print(round(np.mean(lr),2), round(np.std(lr),2))

3.04 1.18


In [ ]:
print(round(np.mean(uw),2), round(np.std(uw),2))

0.58 0.19


# Qualitative

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

device = "cuda"
model_id = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [ ]:
prompt = '(Below are ONLY lyrics from a rap song)\n'
torch.manual_seed(100)
for start in first_lines[:10]:
  input_ids = tokenizer([prompt + start], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  o = o.replace(prompt, '')
  print(o)
  print('=========')

godfathers in the house
and the house in me (above), I say it's too cold
I don't let nothing stay to me
the house in me, I can hear them calling in my chest
if I get lost on thursday (the day of the soul), he's there, right in my sights
the house in me, I'm in the mood for love
the house in me, so cold so
tell me what ya
what ya say nah
and he says hey
but the nigga tells me to wait
I ain't play nah
I ain't play wit dat nigga
I ain't play wit dat nigga
So I'm waiting
wait for tha sky to fall
wait for tha sky to fall
I don't gots no reason to keep
I don't gots no reason to keep

cut the music up
Just wait til the world stops
But wait till the world stops
I'm gonna tell you what's in my life,
How I made more than a hundred thousand dollars a week
I've been here before so many times
And my life is a thousand miles long
No no no, it's not like that,
I can't wait until I get out the door
Well, I'll be back
 calling all hustlers calling all players
calling all hustlers call
call all hustlers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)

In [ ]:
prompt = '(Below are ONLY lyrics from a rap song)\n\n'
for start in first_lines[:10]:
  input_ids = tokenizer([prompt + start], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  o = o.replace(prompt, '')
  print(o)
  print('=========')

godfathers in the house
The two have a heart
the two have never talked

a true friendship and love
the two have never been lonely
And all that he cared for
A love has never been lost
A romantic home in my back yard
My mother had a heart
The two have never had that love
A home in my back yard
This year the two have two great children. They're also
tell me what ya
Tell me what ya
Tell me what you know
Tell me who's going to get your money, please
Tell me what you aren't
Tell me how big
Tell me who's going to buy your shit
Tell me how big
Tell me what you haven't
Tell me who's going to buy your shit
Tell me who's going to buy your shit
Tell me who's looking at
cut the music up
with the lyrics written by Eric Clapton as well as the lyrics written by Michael Jackson in the band The Cure.

The lyrics were posted on this page to the internet in an attempt to explain why Eric Clapton, the son and mentor of the rap artists, is not doing his best. But if you know him, here's how.
The source of t

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", use_fast=True)
model = AutoModelForCausalLM.from_pretrained('mydata/MyDrive/mlp/model').to(device)

In [ ]:
for start in first_lines[:10]:
  input_ids = tokenizer([start], return_tensors="pt").to(device).input_ids
  sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id
  )

  o = tokenizer.decode(sample_output[0], skip_special_tokens=True)
  print(o)
  print('=========')

godfathers in the house
and one day he gets a message from my brother named ron brown
i know you feel my pain
and they say im coming with the plan to make it so easy by
im leaving but you know i gotta give you a chance
i say i aint going down down but im looking up to my man
so i keep it good to see you smile in the night
and i love your smile
and i cant stop my tears
and i
tell me what ya
what you wanna do when you can go to sleep
oh oh oh oh
you know what i want
how you wanna
what you wanna do when you can go to sleep
oh oh oh oh oh yeah yeahi am an artist with a mission that makes me feel like im trapped in a cage
cause all the time i got to go and i be alone
theres a way that everybody knows
and i wonder where im going because im stuck in a cage
cut the music up
the beats is loud im sure to send them higher
dont go to sleep youre in the mind of the lord
youre not the one to see the god within within
and all this noise around my head youre the chosen
youre not the one to see me fall